In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
df = pd.read_csv('IMDb Movies India.csv')


In [8]:
df.head()


,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [10]:
df.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [11]:
df.describe()

,Rating
count,7919.000000
mean,5.841621
std,1.381777
min,1.100000
25%,4.900000
50%,6.000000
75%,6.800000
max,10.000000


In [12]:
df = df.dropna(subset=['Rating'])


In [13]:
df['Year'] = df['Year'].astype(str)
df['Year'] = df['Year'].str.replace(r'[()]', '', regex=True)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Year'] = df['Year'].fillna(df['Year'].median())

In [14]:
df['Duration'] = df['Duration'].astype(str)

df['Duration'] = df['Duration'].str.replace(' min', '', regex=False)

df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

df['Duration'] = df['Duration'].fillna(df['Duration'].median())

In [15]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_genres = encoder.fit_transform(df[['Genre']])

In [16]:

X = df[['Year', 'Duration', 'Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']]
y = df['Rating']
X_num = X[['Year', 'Duration']].values


In [17]:
encoder_genre = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_director = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor1 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor2 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor3 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


In [18]:
X_genre = encoder_genre.fit_transform(X[['Genre']])
X_director = encoder_director.fit_transform(X[['Director']])
X_actor1 = encoder_actor1.fit_transform(X[['Actor 1']])
X_actor2 = encoder_actor2.fit_transform(X[['Actor 2']])
X_actor3 = encoder_actor3.fit_transform(X[['Actor 3']])

In [19]:
X_final = np.hstack([X_num, X_genre, X_director, X_actor1, X_actor2, X_actor3])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))